In [2]:
import torch
from BranchDetection.branch_detection_net import BranchDetector
from BranchDetection.branch_detection_utils import load_checkpoint, predict_sidebranches
import os
from temporal_registration import temporal_registration # Do I need to import other functions this depends on as well?

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



## Branch detection

In [3]:
DEVICE = torch.device("cpu")
CHECKPOINT_FILE = "/store/DAMTP/cr661/OCTImaging/checkpoints/lr0001"
IMG_PATH = "/store/DAMTP/cr661/OCTImaging/data/5_Annotated_Files/SplittedImages"
IMG_NAME = "IBIS4_453-636-202_LAD_NI_B_BL_2.png"

In [4]:
! export CUDA_VISIBLE_DEVICES=1

In [5]:
# Initialise network
branch_detector = BranchDetector(num_blocks=1, spatial_res=1024).to(DEVICE)
branch_detector, _, _, _ = load_checkpoint(
    checkpoint=CHECKPOINT_FILE,
    model=branch_detector,
    learning_rate=0,
    device=DEVICE,
)

/alt/applic/user-maint/cr661/conda_envs/oct/lib/python3.7/site-packages/torch/cuda/__init__.py:120: UserWarning: 
    Found GPU%d %s which is of cuda capability %d.%d.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is %d.%d.
    
  warnings.warn(old_gpu_warn.format(d, name, major, minor, min_arch // 10, min_arch % 10))


In [6]:
# Prediction of sidebranches via trained neural network
idx, slide_num, prediction = predict_sidebranches(
    branch_detector=branch_detector,
    img_path=IMG_PATH,
    img_name=IMG_NAME,
    device=DEVICE,
)

/mhome/damtp/t/cr661/Projects/OCTimaging/BranchDetection/branch_detection_utils.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_pred = F.softmax(input=prediction)


## Dynamic Time Warping

In [7]:
patient_id = "453-636-202"
artery = "LAD"
measurements = ["BL", "FUP"]  # must be 2

predictions = [None] * 2

for i, m in enumerate(measurements):

    filenames = next(os.walk(IMG_PATH), (None, None, []))[2]
    imgs_of_interest = [
        fn for fn in filenames if (patient_id in fn and artery in fn and m in fn)
    ]
    predictions_m = [None] * len(imgs_of_interest)

    for img_name in imgs_of_interest:
        _, slice, pred = predict_sidebranches(
            img_name=img_name,
            branch_detector=branch_detector,
            img_path=IMG_PATH,
            device=DEVICE,
        )
        predictions_m[slice - 1] = pred[1]

    predictions[i] = predictions_m

temp_reg = temporal_registration(
    t1=predictions[0],
    t2=predictions[1],
    smoother="moving_median",
    smoothing_window=7,
    discr_cutoff=0.9,
)

temp_reg_ind = [temp_reg[3], temp_reg[4]]
# temp_reg[3] and temp_reg[4] are integer lists of the same length, giving corresponding slice indices for "BL" and "FUP" images at the same position within the lists.